In [ ]:
import glob
import os
import subprocess

def convert(n, d):
    get_ipython().system("jupyter nbconvert {} --to markdown --output {}".format(n, d))
    #subprocess.check_call(["jupyter", "nbconvert", n, "--to", "markdown", "--output", d])

highlevel_dirs = ["tools"] # + glob.glob("./sem*")

for subdir in highlevel_dirs:
    notebooks = glob.glob(subdir + "/*.ipynb")
    print(subdir, notebooks)
    for m in glob.glob(subdir + "/*.md"):
        os.remove(m)
    if len(notebooks) == 1:
        convert(notebooks[0], "README")
    else:
        for n in notebooks:
            convert(n, os.path.basename(n.replace(".ipynb", "")))
        


tools ['tools/set_up_magics.ipynb', 'tools/set_up_magics_dev.ipynb']


In [ ]:
" ".join(['jupyter', 'nbconvert', './sem04-asm-arm/arm.ipynb', '--to', 'pdf', '--output', 'arm'])

In [11]:
import re

def basic_improve(fname):
    with open(fname, "r") as f:
        r = f.read()
    for b in ["\x00", "\x1B", "\x08"]:
        r = r.replace(b, "")
    with open(fname, "w") as f:
        f.write(r)
    get_ipython().system("dos2unix {}".format(fname))

def improve_md(fname):
    with open(fname, "r") as f:
        r = f.read()
    r = r.replace("```python\n%%cpp", "```cpp\n%%cpp")
    r = r.replace('\n', "SUPER_SLASH" + "_N_REPLACER")
    r = re.sub(r'\<\!--MD_BEGIN_FILTER--\>.*?\<\!--MD_END_FILTER--\>', "", r)
    r = re.sub(r'(\#SET_UP_MAGIC_BEGIN.*?\#SET_UP_MAGIC_END)', "<too much code>", r)
    r = r.replace("SUPER_SLASH" + "_N_REPLACER", '\n')
    
    def file_repl(matchobj, path=os.path.dirname(fname)):
        fname = os.path.join(path, matchobj.group(1))
        if not fname.endswith("__FILE__"):
            with open(fname, "r") as f:
                return "\n```\n" + f.read() + "\n```\n"
    
    r = r.replace("</td>", "")
    r = r.replace("</tr>", "")
    
    r = re.sub(r'\<\!--MD_FROM_FILE (.*?) --\>', file_repl, r)
    with open(fname, "w") as f:
        f.write(r)
        
def improve_file(fname):
    basic_improve(fname)
    if fname.endswith(".md"):
        improve_md(fname)


In [13]:
for sfx in [".ipynb", ".md"]:
    for hdir in highlevel_dirs:
        for fname in glob.glob("./{}/*".format(hdir) + sfx):
            improve_file(fname)

dos2unix: converting file ./tools/set_up_magics.ipynb to Unix format ...
dos2unix: converting file ./tools/set_up_magics_dev.ipynb to Unix format ...
dos2unix: converting file ./tools/set_up_magics_dev.md to Unix format ...


FileNotFoundError: [Errno 2] No such file or directory: './tools/./interactive_launcher_tmp/3803772146190624.log'

In [ ]:
for subdir in highlevel_dirs:
    for sfx in [".ipynb", ".md", ".c", ".cpp"]:
        get_ipython().system("git add {}/*{}".format(os.path.basename(subdir), sfx))
    get_ipython().system("git add -f {}/bash_popen_tmp/*.html".format(os.path.basename(subdir)))
get_ipython().system("git add -u")
get_ipython().system("git commit -m 'yet another update'")
get_ipython().system("git push origin master")